In [1]:
!uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

^C


Using Python 3.12.10 environment at: E:\MinhHoang\LLMOps\.venv
Resolved 74 packages in 1m 07s
Prepared 60 packages in 4m 10s
Uninstalled 1 package in 320ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 69 packages in 1.66s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.11.0
 + attrs==25.4.0
 + certifi==2025.10.5
 + charset-normalizer==3.4.4
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + flatbuffers==25.9.23
 + frozenlist==1.8.0
 + greenlet==3.2.4
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.3
 + humanfriendly==10.0
 + idna==3.11
 + jiter==0.11.1
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==1.0.3
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 + langchain-core==1.0.2
 + langchai

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_community.document_loaders import TextLoader

In [9]:
loader = TextLoader("E:/MinhHoang/LLMOps/data/agentic_ai_overview.txt", encoding="utf-8")
document = loader.load()

In [10]:
document

[Document(metadata={'source': 'E:/MinhHoang/LLMOps/data/agentic_ai_overview.txt'}, page_content='AGENTIC AI: THE NEXT FRONTIER IN ARTIFICIAL INTELLIGENCE\n\nIntroduction\n\nAgentic AI represents a paradigm shift in how we think about and deploy artificial intelligence systems. Unlike traditional AI that responds to prompts or follows predetermined rules, agentic AI systems possess the ability to set goals, make decisions, take actions, and adapt their behavior based on environmental feedback. This autonomous capability marks a significant evolution from passive AI assistants to proactive AI agents that can operate with varying degrees of independence.\n\nThe term "agentic" comes from the concept of agency—the capacity to act independently and make choices. In the context of AI, this means systems that can break down complex tasks, plan multi-step solutions, use tools and resources, and persist toward goals even when facing obstacles or unexpected situations. As we stand at the cusp of 

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=100
)

In [14]:
text_chunk = text_splitter.split_documents(document)

In [15]:
text_chunk

[Document(metadata={'source': 'E:/MinhHoang/LLMOps/data/agentic_ai_overview.txt'}, page_content='AGENTIC AI: THE NEXT FRONTIER IN ARTIFICIAL INTELLIGENCE\n\nIntroduction'),
 Document(metadata={'source': 'E:/MinhHoang/LLMOps/data/agentic_ai_overview.txt'}, page_content='Agentic AI represents a paradigm shift in how we think about and deploy artificial intelligence systems. Unlike traditional AI that responds to prompts or follows predetermined rules, agentic AI systems possess the ability to set goals, make decisions, take actions, and adapt their behavior based on environmental feedback. This autonomous capability marks a significant evolution from passive AI assistants to proactive AI agents that can operate with varying degrees of independence.'),
 Document(metadata={'source': 'E:/MinhHoang/LLMOps/data/agentic_ai_overview.txt'}, page_content='The term "agentic" comes from the concept of agency—the capacity to act independently and make choices. In the context of AI, this means system

In [16]:
! uv pip install faiss-cpu

^C


In [18]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [19]:
embeddings = OpenAIEmbeddings()

In [20]:
vector_store = FAISS.from_documents(text_chunk, embeddings)

In [21]:
vector_store

In [30]:
retriever=vector_store.as_retriever()

In [23]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vector_store.similarity_search(query, k=3)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)

Document 1:
Agentic AI represents a paradigm shift in how we think about and deploy artificial intelligence systems. Unlike traditional AI that responds to prompts or follows predetermined rules, agentic AI systems possess the ability to set goals, make decisions, take actions, and adapt their behavior based on environmental feedback. This autonomous capability marks a significant evolution from passive AI assistants to proactive AI agents that can operate with varying degrees of independence.
--------------------------------------------------
Document 2:
Defining Agentic AI

Agentic AI refers to artificial intelligence systems that demonstrate autonomous behavior in pursuit of goals. These systems go beyond simple input-output operations by exhibiting several key characteristics:

Goal-directed behavior: Agentic AI systems work toward defined objectives, whether explicitly programmed or learned through interaction.

Planning and reasoning: They can decompose complex problems into mana

In [24]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [25]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [26]:
from langchain_core.output_parsers import StrOutputParser

In [27]:
output_parser=StrOutputParser()

In [29]:
from langchain_openai import ChatOpenAI

llm_model=ChatOpenAI(model_name="gpt-4.1-mini")

In [31]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [32]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI refers to artificial intelligence systems capable of autonomous, goal-directed behavior. Unlike traditional AI that passively responds to prompts or follows fixed rules, agentic AI can set goals, make decisions, take actions, and adapt based on environmental feedback. These systems demonstrate planning and reasoning by breaking down complex tasks into sub-tasks and formulating strategies to solve them. They can also use tools and resources and persist toward their objectives despite obstacles or unexpected situations. This represents a shift from reactive AI to proactive agents that operate with varying degrees of independence. Agentic AI promises to enhance productivity, enable new applications, and augment human capabilities across many domains. Understanding this paradigm is crucial for businesses, developers, and society as AI evolves. The concept is rooted in agency—the capacity to act independently and make choices.'